#  Setup & Imports

In [1]:
import pandas as pd
import cx_Oracle
import logging
import os
import sys
sys.path.append(os.path.abspath("../src"))
from oracle_db_manager import OracleDBManager
from dotenv import load_dotenv


# Load Cleaned Data

In [2]:
df = pd.read_csv('../data/bank_reviews_clean.csv')
df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
df.head()

,review,rating,date,bank,source
0,👌👍,5,2025-06-04,Commercial Bank of Ethiopia,Google Play Store
1,very niec,5,2025-06-04,Commercial Bank of Ethiopia,Google Play Store
2,best app of finance,5,2025-06-04,Commercial Bank of Ethiopia,Google Play Store
3,yetemeta,1,2025-06-03,Commercial Bank of Ethiopia,Google Play Store
4,Engida Kebede Fetera,5,2025-06-03,Commercial Bank of Ethiopia,Google Play Store


# Load .env variables

In [3]:
load_dotenv()

True

#  -- Initialize and Connect to Oracle --

#### Create DB instance using environment variables

In [4]:
# Initialize database connection
user = os.getenv("ORACLE_USER")
password = os.getenv("ORACLE_PASSWORD")
dsn = os.getenv("ORACLE_DSN")

        # Determine connection mode based on the user (as discussed previously)
connection_mode = None
        # You'll need cx_Oracle imported here if you don't already have it
        # For simplicity, assuming cx_Oracle is available for cx_Oracle.SYSDBA
try:
    import cx_Oracle
    if user and user.upper() == "SYS":
        connection_mode = cx_Oracle.SYSDBA
except ImportError:
    logging.warning("cx_Oracle not imported. Cannot set SYSDBA mode if user is SYS.")

logging.info(f"Attempting to connect with user: {user}, dsn: {dsn}")

        # Pass the mode to the OracleDBManager constructor
db = OracleDBManager(user=user, password=password, dsn=dsn, mode=connection_mode)
db.connect()

Attempting to connect to DSN: localhost:1521/xepdb1 as user: FINTECH_APP_USER
Database connection established.


# Create Tables (Run Once Only)

In [5]:
# Run this only once, or comment out after tables are created
try:
    db.create_tables()
except Exception as e:
    print("⚠️ Tables may already exist:", e)

Creating banks table...
Banks table created successfully.
Reviews table already exists, skipping creation.


#  -------- Insert Banks -------- 

In [6]:
bank_names = df['bank'].unique()
bank_id_map = db.insert_banks(bank_names)
bank_id_map

Inserted/updated 3 banks.


{'Commercial Bank of Ethiopia': 1, 'Bank of Abyssinia': 2, 'Dashen Bank': 3}

#  -------- Insert Reviews -------- 

In [7]:
db.insert_reviews(df, bank_id_map)

Inserted 1106 reviews.


# -------- Verify in Oracle -------- 

In [8]:
db.connect()
db.cursor.execute("SELECT COUNT(*) FROM reviews")
print("Review count:", db.cursor.fetchone()[0])

Attempting to connect to DSN: localhost:1521/xepdb1 as user: FINTECH_APP_USER
Database connection established.
Review count: 1106


# ---- Close Connection ---- 

In [9]:
db.close()

Cursor closed.
Database connection closed.
